In [1]:
!pip install eli5

In [11]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance


Wczytywanie danych

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix2/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix2/dw_matrix_car


In [8]:
!ls

data  LICENSE  M2_day2.ipynb  README.md


In [0]:
df = pd.read_hdf('data/car.h5')

In [5]:
df.shape

(106494, 155)

In [6]:
df.columns.values

array(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       'feature_kurtyny-powietrzne', 'feature_klimatyzacja-dwustrefowa',
       'feature_światła-led', 'feature_czujnik-zmierzchu',
       'feature_elektrycznie-ustawiane-lusterka',
       'feature_asr-(kontrola-trakcji)',
       'feature_poduszka-powietrzna-kierowcy', 'feature_cd',
       'feature_elektryczne-szyby-przednie',
       'feature_poduszka-powietrzna-pasażera',
       'feature_system-start-stop', 'feature_światła-do-jazdy-dziennej',
       'feature_komputer-pokładowy', 'feature_elektryczne-szyby-tylne',
       'feature_klimatyzacja-manualna', 'feature_tapicerka-welurowa',
       'feature_czujnik-deszczu', 'feature_światła-przeciwmgielne',
       'feature_ogrzewanie-postojowe', 'feature_radio-niefabryczne',
       'feature_regulowane-zaw

Dummy model

In [7]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [10]:
feats = ['car_id']
X = df[feats].values
y=df['price_value']

model = DummyRegressor()
model.fit(X,y)
y_pred = model.predict(X)
mae(y,y_pred)

39465.934630440985

In [11]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [13]:
df['price_currency'].value_counts(normalize = True) * 100

PLN    99.80844
EUR     0.19156
Name: price_currency, dtype: float64

In [0]:
df = df[df['price_currency'] !='EUR']

In [15]:
df.shape

(106290, 155)

Features

In [0]:
suffix_cat = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if suffix_cat in feat:
    df[feat] = factorized_values
  else:
    df[feat + suffix_cat] = factorized_values
 

In [22]:
cat_feats = [x for x in df.columns if suffix_cat in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X,y, cv=3, scoring='neg_mean_absolute_error')

In [24]:
np.mean(scores)

-19566.588937368328

In [27]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X,y)

imp = PermutationImportance(m).fit(X,y)
eli5.show_weights(imp,feature_names=cat_feats)

Weight,Feature
0.2548 ± 0.0085,param_napęd__cat
0.2047 ± 0.0052,param_faktura-vat__cat
0.1948 ± 0.0063,param_stan__cat
0.1458 ± 0.0049,param_rok-produkcji__cat
0.0627 ± 0.0023,param_moc__cat
0.0417 ± 0.0002,feature_kamera-cofania__cat
0.0415 ± 0.0030,param_skrzynia-biegów__cat
0.0283 ± 0.0030,param_marka-pojazdu__cat
0.0193 ± 0.0016,param_pojemność-skokowa__cat
0.0164 ± 0.0002,feature_bluetooth__cat
